这次要抓取的目标是[豆瓣电影 TOP250](https://movie.douban.com/top250)，主要是要的是 BeautifulSoup，一道美味的汤啊~ 🍵

# [](https://hoxis.github.io/python-douban-top250.html#%E8%A7%A3%E6%9E%90 "解析")解析

其实简单的网络爬虫无外乎查看网页源码，从源码中获取自己想要的东西，然后对其进行处理。

[![豆瓣电影 Top 250](https://www.github.com/hoxis/token4md/raw/master/1528077947010.jpg)](https://www.github.com/hoxis/token4md/raw/master/1528077947010.jpg)

通过查看页面元素代码可以看出：

1.  电影条目是被 `<ol class="grid_view">` 所包围的；
2.  其中每个电影条目是一个 `<li>`；
3.  另外，每页有 25 个条目，共 10 页，这意味着需要解析多页数据。

我们来看下其中一个条目的源码：

```html

    <li>  
    	<div class="item">  
    		<div class="pic">  
    			<em class="">1</em>  
    			<a href="https://movie.douban.com/subject/1292052/">  
    				<img width="100" alt="肖申克的救赎" src="https://img3.doubanio.com/view/photo/s_ratio_poster/public/p480747492.webp" class=""></a>  
    		</div>  
    		<div class="info">  
    			<div class="hd">  
    				<a href="https://movie.douban.com/subject/1292052/" class="">  
    					<span class="title">肖申克的救赎</span>  
    					<span class="title">&nbsp;/&nbsp;The Shawshank Redemption</span>  
    					<span class="other">&nbsp;/&nbsp;月黑高飞(港) / 刺激1995(台)</span></a>  
    				<span class="playable">[可播放]</span></div>  
    			<div class="bd">  
    				<p class="">导演: 弗兰克·德拉邦特 Frank Darabont&nbsp;&nbsp;&nbsp;主演: 蒂姆·罗宾斯 Tim Robbins /...  
    					<br>1994&nbsp;/&nbsp;美国&nbsp;/&nbsp;犯罪 剧情</p>  
    				<div class="star">  
    					<span class="rating5-t"></span>  
    					<span class="rating_num" property="v:average">9.6</span>  
    					<span property="v:best" content="10.0"></span>  
    					<span>1041580人评价</span></div>  
    				<p class="quote">  
    					<span class="inq">希望让人自由。</span></p>  
    				<p>  
    					<span class="gact">  
    						<a href="https://movie.douban.com/wish/50494322/update?add=1292052" target="_blank" class="j a_collect_btn" name="sbtn-1292052-wish" rel="nofollow">想看</a></span>&nbsp;&nbsp;</p>  
    			</div>  
    		</div>  
    	</div>  
    </li>  
    </pre>
```

对于每个条目，我们需要解析出其中的 电影名称、评分、评价人数，及一句话点评。

## [](https://hoxis.github.io/python-douban-top250.html#%E6%A0%87%E9%A2%98%E3%80%81%E8%AF%84%E5%88%86%E3%80%81%E8%AF%84%E4%BB%B7%E8%A7%A3%E6%9E%90 "标题、评分、评价解析")标题、评分、评价解析

标题是在 `肖申克的救赎` 里的，我们可以使用

```
    find("span", attrs={"class": "title"}).getText()  
```

获取到，但是明显这里又多个 `<span class="title">`，那我们就只获取第一个，其他的不关心。

评分和评价的解析和标题类似，用同一种解析方法解析即可。

## [](https://hoxis.github.io/python-douban-top250.html#%E8%AF%84%E4%BB%B7%E4%BA%BA%E6%95%B0%E8%A7%A3%E6%9E%90 "评价人数解析")评价人数解析

评价人数这里是这样的： `1041580人评价`，明显跟上面的不同，它没有 class 属性，这里只能通过 text 来查找了：

```
    find(text=re.compile('人评价$'))  
```

这里用了正则表达式来进行匹配，即匹配以 `人评价` 结尾的文本。

## [](https://hoxis.github.io/python-douban-top250.html#%E4%B8%8B%E4%B8%80%E9%A1%B5%E8%A7%A3%E6%9E%90 "下一页解析")下一页解析

```
    <span class="next">  
        <link rel="next" href="?start=25&amp;filter="/>  
        <a href="?start=25&amp;filter=" >后页&gt;</a>  
    </span>  
```

对照代码，需要从中解析出下一页的连接 `?start=25&amp;filter=`。

解析方法类似于标题的解析，先解析出 `<span class="next">`，然后解析其中的 `<a>` 标签。

```
    find("span", attrs={"class":"next"}).find("a")  
```

要解析的东西基本上就是这些，最后需要将解析结果保存到文件。

# [](https://hoxis.github.io/python-douban-top250.html#%E5%AE%9E%E7%8E%B0%E4%BB%A3%E7%A0%81 "实现代码")实现代码

In [1]:
# encoding:utf-8

import requests, re
from bs4 import BeautifulSoup

def download_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "lxml")
    return soup

def parse_soup(soup):
    return_list = []
    # 利用 class 属性找到 grid
    grid = soup.find("ol", attrs={"class": "grid_view"})
    # 不加 attrs= 也可以
    # grid = soup.find("ol", {"class": "grid_view"})
    if grid:
        # 利用标签获取 list
        movie_list = grid.find_all("li")

        # 遍历 list
        for movie in movie_list:
            # 一个电影有多个名字，这里只取第一个
            # getText() 方法获取到值
            title = movie.find("span", attrs={"class": "title"}).getText()
            # print(title)
            rating_num = movie.find("span", attrs={"class": "rating_num"}).getText()
            inq = movie.find("span", attrs={"class": "inq"})
            # 利用 text 配合正则表达式匹配搜索文本
            rating_p = soup.find(text=re.compile('人评价$'))
            # 有些暂时没有一句话评论
            if not inq:
                inq = "暂无"
            else:
                inq = inq.getText()
            return_list.append(title + "，" + rating_p + "，评分：" + rating_num + "，一句话评价：" + inq)

        next_page = soup.find("span", attrs={"class":"next"}).find("a")
        if next_page:
            return return_list, next_page["href"]
        else:
            return return_list, None

if __name__ == "__main__":
    url = "https://movie.douban.com/top250"
    next_url = ""
    # 将结果保存到文件
    with open("doubanMoviesTop250.txt","w+") as f: 
        while next_url or next_url == "":
            soup = download_page(url + next_url)
            movie_list, next_url = parse_soup(soup)
            # 将 list 拆分成不同行
            f.write("\n".join(movie_list))

![](http://ww3.sinaimg.cn/large/006tNc79ly1g53v4gz6x6j315m0ss4bz.jpg)